In [5]:
import matplotlib
matplotlib.use('Agg')
# Remove padding around screen

from IPython.core.display import display, HTML
import matplotlib.pyplot as plt
import matplotlib.image as img

from os import walk
import tensorflow as tf,numpy,keras,PIL
from PIL import Image

import sys
from os.path import dirname
sys.path.append("/home/jason/sproj/donkeycar")

import donkeycar as dk
import donkeycar.parts.datastore as datastore

In [7]:
a = datastore.Tub("../datasets/8track/data-4-4/data/tub_5_18-04-04/")

path_in_tub: ../datasets/8track/data-4-4/data/tub_5_18-04-04/
Tub exists: ../datasets/8track/data-4-4/data/tub_5_18-04-04/


In [8]:
a.get_num_records()

1196